In [1]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import matplotlib.pyplot as plt
from scipy.linalg import hadamard
import statistics


In [2]:
num_user=[1000, 10000, 100000, 1000000]
all_d = [32, 64, 128, 256, 512]
num_repeats = 20

In [3]:
def naive_sum(I, phi, tau):
    
    norm=np.linalg.norm(I, ord=2, axis=1, keepdims=False)
    trunc_vector = tau/norm
    trunc_vector[trunc_vector>1] = 1
    trunc_vector = np.reshape(trunc_vector, [-1,1])
    I_truc = np.multiply(I, trunc_vector)
    trunc_sum = np.sum(I_truc, axis=0, keepdims=False)
    noise_scale  = np.sqrt(np.sum(1/(2*phi)))
    noisy_sum = trunc_sum + np.random.normal(0, noise_scale, size = d)*tau
#     acc = np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm)
#     print(acc)
    return noisy_sum

In [7]:
#different users
times = []
relative_error = []
real_results = []
for i in range(len(all_d)):
    n = num_user[2]
    B = 1000000
    d = all_d[i]
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = int(n*fh)
    
    #initialize eps and data
    phi_l = np.random.uniform(1/(10*n), 100/n, nl)
    phi_h = np.random.uniform(10, 100, nh)
    phi = np.append(phi_l, phi_h)
    phi_max = max(phi)
    phi_min = min(phi)
#     I = np.random.normal(800, 20, size =(n,d) )
    I = np.random.randint(0, 1000, size =(n,d) )
    I[I<0]=1
    I = np.int_(I)
    I = np.float32(I)
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    err = 0

    
  
    start = time.time()
    for j in range(num_repeats):
        noisy_sum = naive_sum(I, phi, tau = 500*math.sqrt(d))
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)
    real_results.append(l2_norm)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)

print("relative_error")
print(relative_error)
    

times
[0.008617162704467773, 0.009021759033203125, 0.014806985855102539, 0.028853893280029297, 0.06406116485595703]
relative_error
[0.23676260525777265, 0.4053808583211731, 0.5561575202062095, 0.7205463094009348, 0.9353661131286758]


In [8]:
def radius_sum(I, phi, B, beta, d):
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    noisy_sum = np.zeros([t,d])
    each_tau = np.zeros(t)
    cur_min = 100
    for i in range(t):
        tau_i = math.pow(2,i)*np.sqrt(phi/phi_max)
        norm=np.linalg.norm(I, ord=2, axis=1, keepdims=False)
        trunc_vector = tau_i/norm
        trunc_vector[trunc_vector>1] = 1
        trunc_vector = np.reshape(trunc_vector, [-1,1])
        I_truc = np.multiply(I, trunc_vector)
        trunc_sum = np.sum(I_truc, axis=0, keepdims=False)
        noisy_sum[i] = trunc_sum + np.random.normal(0, math.sqrt(n*math.pow(2,2*i)*t/(2*phi_max)), size = d) - math.pow(2,i)*math.sqrt(n*t/(2*phi_max)*math.log(t*d/beta))
        noisy_sum[i][noisy_sum[i]<0]=0
        #l2 error
        each_tau[i] =np.sqrt(np.sum(((true_sum-noisy_sum[i]))**2))/(l2_norm)
        if each_tau[i]<cur_min:
            idx = i
            cur_min = each_tau[i]


    best_sum = noisy_sum[idx]+math.pow(2,idx)*math.sqrt(n*t/(2*phi_max)*math.log(t*d/beta))

#     best_sum = noisy_sum.max(axis=0)
#     best_sum[best_sum<0] = 0
    best_acc = np.sqrt(np.sum(((true_sum-best_sum))**2))/(l2_norm)
#     print(best_acc)

    return best_sum


In [15]:
#different users
times = []
relative_error = []
real_results = []
for i in range(len(all_d)):
    n = num_user[2]
    B = 1000000
    d = all_d[i]
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = n-nl
    
    #initialize eps and data
    phi_l = np.random.uniform(1/(10*n), 100/n, nl)
    phi_h = np.random.uniform(10, 100, nh)
    phi = np.append(phi_l, phi_h)
    phi_max = max(phi)
    phi_min = min(phi)
#     I = np.random.normal(800, 20, size =(n,d) )
    I = np.random.randint(0, 1000, size =(n,d) )
    I[I<0]= -I[I<0]
    I = np.int_(I)
    I = np.float32(I)
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    err = 0

  
    start = time.time()
    for j in range(num_repeats):
        noisy_sum = radius_sum(I, phi, B, beta, d)
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)

    real_results.append(l2_norm)

    
print("times")
print(times)

print("relative_error")
print(relative_error)

times
[0.27681779861450195, 0.46712207794189453, 0.9041337966918945, 1.9234790802001953, 3.8947548866271973]
relative_error
[0.06405113734517154, 0.097656713435242, 0.06796255117550778, 0.10423024044541647, 0.1976921034874462]


In [10]:
def random_hadamard(d,t=1):
    H = hadamard(d)
    #print(np.ones((d,1))))
    #print(np.matmul(H,np.transpose(H)))
    rand = np.random.uniform(low=0,high=1,size=d)
    flipped = (rand < 0.5)
    diagonal = np.ones(d)
    diagonal[flipped] = -1
    D = np.diag(diagonal)
    HD = np.matmul(H,D)
    # rotate t-1 more times
    for i in range(t-1):
        rand = np.random.uniform(low=0,high=1,size=d)
        flipped = (rand < 0.5)
        diagonal = np.ones(d)
        diagonal[flipped] = -1
        D = np.diag(diagonal)
        #print(D)
        HD = np.matmul(HD,np.matmul(H,D))
    #print(HD)
    #print(np.matmul(HD,np.transpose(HD)))
    return HD  

In [11]:
def median_selection(column, phi, B, beta):
    T = int(math.log(B, 2))+2
    trimmed = column[nl:nh]
    left = -B
    right = B
    m = nh/2
    i = 0
    while i < T:
        mid = (left+right)/2
        count = len([xi for xi in column if xi<=mid])
        count_noisy = count+ np.random.normal(0, np.sqrt(nh*T/(2*10)))
        if count_noisy <= m:
            left = mid + 1
        else:
            right = mid
        i = i+1
    return int((left+right)/2)

In [12]:
def diam_sum(I, phi, B, beta, d):
    n = len(phi)
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
    HD = random_hadamard(d)
    HD_inv = np.transpose(HD)
    I_t = np.transpose(I)
    rotat_I_t = np.matmul(HD, I_t)
    rotat_I = np.transpose(rotat_I_t)
    #find median
    m = np.zeros(d)
    for i in range(d):
        m[i] = median_selection(rotat_I[ : ,i], phi/(2*d), d*B, beta/(4*d))
        
#     print(np.matmul(HD_inv,np.transpose(m) )/d)
    #shift and divide into two parts
    shift_rotate_I = rotat_I-m
    positive = copy.deepcopy(shift_rotate_I)
    negative = -copy.deepcopy(shift_rotate_I)
    positive[positive < 0] = 1e-7
    negative[negative < 0] = 1e-7
    pos_sum = radius_sum(positive, phi/4, B*d, beta/4, d)
    neg_sum = radius_sum(negative, phi/4, B*d, beta/4, d)
    
    rotat_res = pos_sum-neg_sum+m*n
    res = np.matmul(HD_inv,np.transpose(rotat_res) )/d
    res[res<0] = 0
    
    pp_sum = np.sum(positive,axis=0, keepdims=False)
    rr = np.matmul(HD_inv,np.transpose(pp_sum) )/d
#     print((l2_norm)/np.sqrt(np.sum(rr**2)))
    
    return res

In [13]:
#different users
times = []
relative_error = []
real_results = []
for i in range(len(all_d)):
    n = num_user[2]
    B = 1000000
    d = all_d[i]
    beta = 0.1
    fl = 0.05
    fh = 0.95
    nl = int(n*fl)
    nh = n-nl
    
    #initialize eps and data
    phi_l = np.random.uniform(1/(10*n), 100/n, nl)
    phi_h = np.random.uniform(10, 100, nh)
    phi = np.append(phi_l, phi_h)
    phi_max = max(phi)
    phi_min = min(phi)
#     I = np.random.normal(800, 20, size =(n,d) )
    I = np.random.randint(0, 1000, size =(n,d) )
    I[I<0]=1
    I = np.int_(I)
    I = np.float32(I)
    t = int(math.log2(math.sqrt(phi_max*d/phi_min)*B))+1
    true_sum = np.sum(I, axis=0, keepdims=False)
    l2_norm = np.sqrt(np.sum(true_sum**2))
  
    start = time.time()
    err = 0
    for j in range(num_repeats):
        noisy_sum = diam_sum(I, phi, B, beta, d)
        err += (np.sqrt(np.sum(((true_sum-noisy_sum))**2))/(l2_norm))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    relative_error.append(err/num_repeats)

    real_results.append(l2_norm)

    
print("times")
print(times)

print("relative_error")
print(relative_error)

times
[3.713901996612549, 7.6386072635650635, 15.903310775756836, 34.787384033203125, 73.09553408622742]
relative_error
[0.031957834075646216, 0.03908924201958591, 0.028846423237016017, 0.038349535371563374, 0.05092422665214154]
